In [3]:
import argparse
from sklearn.preprocessing import OneHotEncoder
from utils.torch_utils import *
from utils.utils import *
import matplotlib.pyplot as plt
import utils.language_helpers
plt.switch_backend('agg')
import numpy as np
from multi_models import Discriminator_logkm

import pandas as pd

In [79]:
batch_size=16
lr=0.001
num_epochs=2000
seq_len=512
data_dir= "data/new_brenda_substrate_df_data.csv"
hidden=512
sub_len=1024


######### Load data #######
# max_examples = 1e6
# data, charmap, inv_charmap, substate_ecfp, logkm_list = utils.language_helpers.load_dataset_ecfp(
#     max_length=seq_len,
#     max_n_examples=max_examples,
#     data_dir=data_dir
# )

data = pd.read_csv(data_dir)
data.head()

,Unnamed: 0,EC,SUBSTRATE,ORGANISM,UNIPROT,log10_KM,KEGG ID,KEGG reaction ID,Sequence,MW,LogP,ECFP,RDKit FP,MACCS FP
0,26,1.1.1.1,1-Octanol,Geobacillus thermodenitrificans,['A4IP64' 'A4ISB9'],0.588832,C00756,['R02878_f'],MQNFTFRNPTKLIFGRGQIEQLKEEVPKYGKKVLLVYGGGSIKRNG...,130.135765,2.3392,0000000000000000000000000000000001000000000000...,0000000000000000000000000000000000000000000000...,0000000000000000000000000000000000000000000000...
1,27,1.1.1.1,1-Octanol,Homo sapiens,['P00326' 'P08319'],-0.408935,C00756,['R02878_f'],MGTKGKVIKCKAAIAWEAGKPLCIEEVEVAPPKAHEVRIQIIATSL...,130.135765,2.3392,0000000000000000000000000000000001000000000000...,0000000000000000000000000000000000000000000000...,0000000000000000000000000000000000000000000000...
2,114,1.1.1.1,NAD+,Acetobacter pasteurianus,[],-0.659379,C00003,"['R00623_f', 'R00624_f', 'R00754_f', 'R02124_f...",MFAMRLHQPKTPLQWEEVPDPVPGPGEIRVKVLACGVCRTDLHVVD...,664.116398,-3.0159,0000000001000000000000100000000000000000000000...,1110111010111011101011111011111011011100100111...,0000000000000000000000000000010000000010000000...
3,115,1.1.1.1,NAD+,Aeropyrum pernix,['Q9Y9P9'],-3.000000,C00003,"['R00623_f', 'R00624_f', 'R00754_f', 'R02124_f...",MKAARLHEYNKPLRIEDVDYPRLEGRFDVIVRIAGAGVCHTDLHLV...,664.116398,-3.0159,0000000001000000000000100000000000000000000000...,1110111010111011101011111011111011011100100111...,0000000000000000000000000000010000000010000000...
4,117,1.1.1.1,NAD+,Euglena gracilis,['B8QU18'],-0.704468,C00003,"['R00623_f', 'R00624_f', 'R00754_f', 'R02124_f...",MKAAVVEQFGKPLAIREVPVPEPGYGQVLIKIIASGVCHTDLHVRD...,664.116398,-3.0159,0000000001000000000000100000000000000000000000...,1110111010111011101011111011111011011100100111...,0000000000000000000000000000010000000010000000...


In [80]:
from utils.constants import AMINO_ACID_TO_ID

charmap = AMINO_ACID_TO_ID
inv_charmap = ['0','A','C','D','E','F','G','H','I','K','L','M','N','P','Q','R','S','T','V','W','Y']

In [81]:
model = Discriminator_logkm(len(charmap), seq_len, batch_size, hidden, sub_len)
model.cuda()
model.load_state_dict(torch.load("checkpoint/test_pred/P_weights_780000.pth"))
print("")

In [82]:
table = np.arange(len(charmap)).reshape(-1,1)
one_hot = OneHotEncoder()
one_hot.fit(table)

OneHotEncoder(categories='auto', drop=None, dtype=<class 'numpy.float64'>,
              handle_unknown='error', sparse=True)

In [83]:
n_train_batches = int(len(data)/batch_size)

In [84]:
from utils.constants import NON_STANDARD_AMINO_ACIDS

In [85]:
def tokenize_string(sample):
    return tuple(sample.lower().split(' '))

In [86]:
def load_dataset_ecfp(max_length, max_n_examples, tokenize=False, max_vocab_size=2048, data_dir=''):
    '''Adapted from https://github.com/igul222/improved_wgan_training/blob/master/language_helpers.py'''
    print ("loading dataset...")

    temp_df = pd.read_csv(data_dir)
    temp_df = temp_df.dropna(subset=["ECFP"])
    
    if len(temp_df) > max_n_examples:
        temp_df = temp_df[:max_n_examples]
    
    temp_seq = list(temp_df["Sequence"])
    temp_ectp = list(temp_df["ECFP"])
    temp_logkm = list(temp_df["log10_KM"])

    lines = []
    seqs = []

    for line, ecfp_line, logkm in zip(temp_seq, temp_ectp, temp_logkm):
        line = line.split(" ")[0].strip()
        if ~any(ext in line for ext in NON_STANDARD_AMINO_ACIDS):
            if tokenize:
                line = tokenize_string(line)
            else:
                line = tuple(line)

            if len(line) > max_length:
                line = line[:max_length]
            
            lines.append([line + ( ("0",)*(max_length-len(line)) ), ecfp_line, logkm] )
            seqs.append(line + ( ("0",)*(max_length-len(line)) ))

    charmap = AMINO_ACID_TO_ID
    inv_charmap = ['0','A','C','D','E','F','G','H','I','K','L','M','N','P','Q','R','S','T','V','W','Y']

    filtered_lines = []
    logkm_lists = []
    substrate_ecfp_lists = []
    for line, ecfp_line, logkm in lines:
        filtered_line = []
        for char in line:
            if char in charmap:
                filtered_line.append(char)
            else:
                # convert all characters to '0' if not exist in inv_charmap
                filtered_line.append('0')
        filtered_lines.append(tuple(filtered_line))
        substrate_ecfp_lists.append(ecfp_line)
        logkm_lists.append(logkm)

    print("loaded {} lines in dataset".format(len(lines)))
    # print(charmap, inv_charmap) # {'P': 0, 'A': 1, 'T': 2, 'G': 3, 'C': 4} ['P', 'A', 'T', 'G', 'C']
    return filtered_lines, charmap, inv_charmap, substrate_ecfp_lists, logkm_lists

max_examples = 1e6
data, charmap, inv_charmap, substate_ecfp, logkm_list = load_dataset_ecfp(
    max_length=seq_len,
    max_n_examples=max_examples,
    data_dir=data_dir
)

loading dataset...
loaded 12524 lines in dataset


In [87]:
model.eval()
seqs = []
substrates = []
logkm_bases = []
logkm_preds = []

for idx in range(n_train_batches):
    _data = np.array(
        [[charmap[c] for c in l] for l in data[idx*batch_size:(idx+1)*batch_size]],
        dtype='int32'
    )
    data_one_hot = one_hot.transform(_data.reshape(-1, 1)).toarray().reshape(batch_size, -1, len(charmap))
    real_data = torch.Tensor(data_one_hot)
    real_data = to_var(real_data)

    logkm_data = torch.Tensor(np.array(logkm_list[idx*batch_size:(idx+1)*batch_size]))
    logkm_data = to_var(logkm_data)

    substrate_data = torch.Tensor(np.array(
        [[c for c in l] for l in substate_ecfp[idx*batch_size:(idx+1)*batch_size]],
        dtype="int32"
    ))
    substrate_data = to_var(substrate_data)

    logkm_pred = model(real_data, substrate_data)

    #output_data.append([list(map("".join, data[idx*batch_size:(idx+1)*batch_size])), substate_ecfp[idx*batch_size:(idx+1)*batch_size], logkm_data.detach().cpu().numpy(), logkm_pred.detach().cpu().numpy()])

    seqs.append(list(map("".join, data[idx*batch_size:(idx+1)*batch_size])))
    substrates.append(substate_ecfp[idx*batch_size:(idx+1)*batch_size])
    logkm_bases.append(logkm_data.detach().cpu().numpy())
    logkm_preds.append(logkm_pred.detach().cpu().numpy())

In [66]:
#output_data[:5]

In [71]:
#len(output_data[2])

In [88]:
import numpy as np

seqs = np.concatenate(seqs, axis=0)
substrates = np.concatenate(substrates, axis=0)
logkm_bases = np.concatenate(logkm_bases, axis=0)
logkm_preds = np.concatenate(logkm_preds, axis=0)

In [97]:
df = pd.DataFrame(columns=["Sequence", "Substrate_ECFP", "log10_KM", "log10_KM_Pred"])

In [100]:
df["Sequence"] = seqs
df["Substrate_ECFP"] = substrates
df["log10_KM"] = logkm_bases
df["log10_KM_Pred"] = logkm_preds

In [103]:
df.to_csv("pred_km_output.csv")